# MODNet on the PBE, HSE and experimental datasets successively

In this model, we will see if training first the MODNet model on the PBE, HSE and experimental datasets and then, fine tuning the model on both the HSE and experimental datasets, and finally, fine tuning the model on the experimental dataset will improve the results.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import os
import copy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

def MD_append(md,lmd):
    md = copy.deepcopy(md)
    for m in lmd:
        md.df_structure.append(m.df_structure)
        md.df_targets.append(m.df_targets)
        md.df_featurized.append(m.df_featurized)
    return md

In [5]:
md_exp = MODData.load('exp_gap_all')
md_exp.df_targets.columns = ['gap']
md_pbe = MODData.load('pbe_gap.zip')
md_pbe.df_targets.columns = ['gap']
md_hse = MODData.load('hse_gap.zip')
md_hse.df_targets.columns = ['gap']

md_joint = MODData.load('exp_pbe_joint')


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766

2021-06-14 18:07:56,566 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7fc0d46bd460> object, created with modnet version <=0.1.7
2021-06-14 18:08:14,878 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7fc0d46bd580> object, created with modnet version <=0.1.7
2021-06-14 18:08:20,436 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7fc0cda2ceb0> object, created with modnet version 0.1.8~develop
2021-06-14 18:08:22,240 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7fc0d6a693d0> object,

In [6]:
md_joint.df_targets = md_joint.df_targets.drop(columns=['pbe_gap', 'difference'])

In [7]:
md_joint.df_targets

,exp_gap
mp-12699,0.00
mp-559459,3.40
mp-21162,0.00
mp-1306,0.00
mp-15252,0.53
...,...
mp-556541,1.80
mp-2602,1.04
mp-30366,0.00
mp-3718,1.45


In [8]:
import uncertainty_toolbox as uct

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes_ph1 = np.ones(5)
maes_ph2 = np.ones(5)
uncertainties = np.ones(5)
metricss = []
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
         
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    #phase 1
    md = MD_append(train,[md_pbe,md_hse])
    
    ga = FitGenetic(md)
    model = ga.run(size_pop=20, num_generations=10, n_jobs=10)
    
    pred, std = model.predict(test, return_unc=True)
    true = test.df_targets
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('mae_ph1')
    print(mae)
    maes_ph1[i] = mae
    
    # phase2
    md = MD_append(train,[md_hse])
    rlr = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=20, verbose=1, mode="auto", min_delta=0)
    es = EarlyStopping(monitor="loss", min_delta=0.001, patience=300, verbose=1, mode="auto", baseline=None,restore_best_weights=True)
    model.fit(md,lr=0.01, epochs = 1000, batch_size = 64, loss='mae', callbacks=[rlr,es], verbose=0)
    
    pred, std = model.predict(test, return_unc=True)
    true = test.df_targets
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('mae_ph2')
    print(mae)
    maes_ph2[i] = mae
    
        # phase2
    rlr = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=20, verbose=1, mode="auto", min_delta=0)
    es = EarlyStopping(monitor="loss", min_delta=0.001, patience=300, verbose=1, mode="auto", baseline=None,restore_best_weights=True)
    model.fit(train,lr=0.01, epochs = 1000, batch_size = 64, loss='mae', callbacks=[rlr,es], verbose=0)
    
    pred, std = model.predict(test, return_unc=True)
    true = test.df_targets
    metrics = uct.metrics.get_all_metrics(pred['exp_gap'].values, std['exp_gap'].values, true['exp_gap'].values)
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    std = std['exp_gap'].drop(std.index[((std['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    uncertainty = np.abs(std).mean()
    print('MAE =')
    print(mae)
    print('uncertainty =')
    print(uncertainty)
    maes[i] = mae
    uncertainties[i] = uncertainty
    metricss.append(metrics)

2021-06-14 18:08:24,048 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:08:24,127 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:08:24,153 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:08:24,219 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:08:24,259 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:08:24,465 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:08:24,499 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:08:24,636 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:08:24,700 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:08:24,790 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:08:25,695 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7fc03e4aac10> object, created with modnet version 0.1.9
2021-06-14 18

100%|██████████| 100/100 [38:26<00:00, 23.07s/it] 

2021-06-14 18:47:06,715 - modnet - INFO - Loss per individual: ind 0: 0.621 	ind 1: 0.681 	ind 2: 0.639 	ind 3: 0.657 	ind 4: 0.601 	ind 5: 0.607 	ind 6: 0.641 	ind 7: 0.625 	ind 8: 0.627 	ind 9: 0.649 	ind 10: 0.646 	ind 11: 0.595 	ind 12: 0.658 	ind 13: 0.624 	ind 14: 0.713 	ind 15: 0.728 	ind 16: 0.623 	ind 17: 0.638 	ind 18: 0.608 	ind 19: 0.966 	


2021-06-14 18:47:09,949 - modnet - INFO - Generation number 1
2021-06-14 18:47:17,051 - modnet - INFO - Multiprocessing on 10 cores. Total of 48 cores available.


100%|██████████| 100/100 [34:07<00:00, 20.47s/it] 

...

 (3/n) Calculating adversarial group calibration metrics
  [1/2] for mean absolute calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


  0%|          | 0/10 [00:00<?, ?it/s]

  [2/2] for root mean squared calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


100%|██████████| 10/10 [00:05<00:00,  1.92it/s]


 (4/n) Calculating sharpness metrics
 (n/n) Calculating proper scoring rule metrics
**Finished Calculating All Metrics**


===================== Accuracy Metrics =====================
  MAE           0.367
  RMSE          0.775
  MDAE          0.029
  MARPD         137.236
  R2            0.802
  Correlation   0.896
=============== Average Calibration Metrics ================
  Root-mean-squared Calibration Error   0.111
  Mean-absolute Calibration Error       0.081
  Miscalibration Area                   0.082
========== Adversarial Group Calibration Metrics ===========
  Mean-absolute Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.162
     Group Size: 0.56 -- Calibration Error: 0.101
     Group Size: 1.00 -- Calibration Error: 0.081
  Root-mean-squared Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.185
     Group Size: 0.56 -- Calibration Error: 0.136
     Group Size: 1.00 -- Calibration Error: 0.111
==============

In [9]:
maes_ph1.mean()

0.3543464838457022

In [10]:
maes_ph2.mean()

0.373707033235109

In [11]:
maes

array([0.43693347, 0.3993549 , 0.29553638, 0.35409614, 0.36735822])

In [12]:
maes.mean()

0.37065582417176934

In [13]:
uncertainties

array([0.16648124, 0.14064462, 0.10966733, 0.12438768, 0.12282877])

In [14]:
uncertainties.mean()

0.1328019306063652

In [15]:
metricss

[{'accuracy': {'mae': 0.43693347197395244,
   'rmse': 0.9242280382492499,
   'mdae': 0.07902669161558151,
   'marpd': 136.24622227221295,
   'r2': 0.6690793361767605,
   'corr': 0.8327573954342087},
  'avg_calibration': {'rms_cal': 0.18228671626396575,
   'ma_cal': 0.1659237536656892,
   'miscal_area': 0.16759975117746376},
  'adv_group_calibration': {'ma_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
    'adv_group_cali_mean': array([0.46470202, 0.24788729, 0.20871571, 0.2001991 , 0.18966261,
           0.1909317 , 0.18514097, 0.17770543, 0.17479208, 0.16592375]),
    'adv_group_cali_stderr': array([0.03359228, 0.03396076, 0.01718741, 0.00959553, 0.00930923,
           0.01324697, 0.00904154, 0.00423822, 0.00285447, 0.        ])},
   'rms_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0.6

#### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: -7%